In [5]:
import dlt
import requests

@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc


In [7]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant2", 
)

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)



Run started at 2025-07-13 13:09:44.710586+00:00 and COMPLETED in 5.44 seconds with 4 steps.
Step extract COMPLETED in 0.11 seconds.

Load package 1752412185.8379345 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.08 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752412185.8379345 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 4.13 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.11 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/llm-training/dlt/db.qdrant2 location to store data
Load package 1752412185.8379345 is LOADED and contains no failed jobs

Step run COMPLETED in 5.44 seconds.
Pipeline zoomcamp_pipeline load step completed in 4.11 seconds
1 load package(s) were loaded to destination q

In [ ]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")